In [1]:
cd ..

/home/jovyan/work/dsi-plus-2/Capstone


In [2]:
%run __init__.py
%matplotlib inline

In [3]:
indiv_path = 'data/sandp500/individual_stocks_5yr/'
indiv = os.listdir(path = indiv_path)

In [4]:
for my_file in indiv:        
    try:
        read_file_path = indiv_path + my_file
        stock_data = pd.read_csv(read_file_path)
        
        ticker = my_file.split('_')[0]
        stock_data.index = ticker + '/' + stock_data['Date']

        close = stock_data["Close"].values
        epsilon = best_epsilon(close)

        ts = trading_signal(close, epsilon)
        ts_df = pd.DataFrame(ts, index=stock_data.index, columns=['TS'])
        ts_df['epsilon'] = epsilon

        pickle_file_path = 'pickled_data/trading_signals/'+my_file.split('.')[0]+'.pkl'
        ts_df.to_pickle(pickle_file_path)
    except:
        print(my_file, 'failed.')

In [5]:
ts_df.head()

,TS,epsilon
Date,,
ULTA/2012-08-13,0.500000,5.0
ULTA/2012-08-14,0.444444,5.0
ULTA/2012-08-15,0.388889,5.0
ULTA/2012-08-16,0.333333,5.0
ULTA/2012-08-17,0.277778,5.0


In [6]:
tech_path = 'pickled_data/technicals/'
technicals = os.listdir(path = tech_path)

In [7]:
ts_path = 'pickled_data/trading_signals/'

In [17]:
training_df = pd.DataFrame()
for my_file in technicals:
    try:
        tech_file_path = tech_path+my_file
        ts_file_path = ts_path+my_file
        
        feat_df = pd.read_pickle(tech_file_path)
        ticker = my_file.split('_')[0]
        
        feat_df['Date'] = feat_df.index
        feat_df.index = ticker + '/' + feat_df.index.astype('str')
        
        ts_df = pd.read_pickle(ts_file_path)
        
        feat_df['Ticker'] = ticker
        feat_df['TS'] = ts_df.TS.loc[feat_df.index]
        feat_df['epsilon'] = ts_df.epsilon.loc[feat_df.index]
        
        training_df = pd.concat([training_df, feat_df])
    except:
        print(my_file, 'failed.')

BF.pkl failed.
BRK.pkl failed.


In [19]:
training_df.shape

(567868, 45)

In [20]:
training_df.sample(10)

,ADX_14_50,ATR_14,BollingerB_5,Bollinger%b_5,BollingerB_20,Bollinger%b_20,Copp_10,Chaikin,EMA_5,EMA_20,...,SO%d_5,SO%d_20,SO%k,Trix_5,Trix_20,TSI_25_13,Date,Ticker,TS,epsilon
LOW/2015-02-12,0.335235,1.301109,0.020476,0.450822,0.100360,0.789611,0.063645,-5.803964e+05,71.239519,69.755949,...,0.571376,0.587573,0.413333,0.003764,0.002367,0.191576,2015-02-12,LOW,0.321429,1.5
WMB/2013-12-26,0.361300,0.783716,0.049321,0.789104,0.115467,0.920320,0.046231,-5.567645e+05,37.147339,36.155804,...,0.622885,0.550385,0.739130,0.006407,0.000165,0.152303,2013-12-26,WMB,0.434959,3.5
FCX/2013-05-14,0.303317,0.918164,0.056018,0.359246,0.202073,0.785456,0.140270,-1.677302e+05,32.049135,31.185526,...,0.539787,0.546554,0.388889,0.008048,-0.001972,0.064385,2013-05-14,FCX,0.369565,1.5
ANSS/2014-02-03,0.421868,1.528106,0.025465,0.460036,0.138533,0.114910,-0.108885,1.164227e+05,79.164501,82.166658,...,0.485789,0.396926,0.770186,-0.007518,-0.001112,-0.314302,2014-02-03,ANSS,0.500000,2.0
PCG/2013-09-17,0.448148,0.686650,0.025647,0.397561,0.051069,0.317602,-0.045194,-6.727909e+05,41.069593,41.686708,...,0.372809,0.435786,0.038168,-0.000637,-0.002604,-0.369938,2013-09-17,PCG,0.300000,1.0
LUK/2014-04-02,0.280902,0.531908,0.084747,0.721872,0.088897,0.813663,0.014448,2.251270e+05,27.504296,27.238725,...,0.610629,0.535991,0.529412,0.003342,-0.000841,-0.021464,2014-04-02,LUK,0.555556,1.5
WRK/2016-08-05,0.304950,1.391025,0.098262,0.841138,0.160761,0.757440,0.078803,4.081951e+04,42.602933,41.880218,...,0.541129,0.556330,0.503906,-0.000720,0.002242,0.158694,2016-08-05,WRK,0.312500,1.5
ES/2015-05-15,0.191657,0.935511,0.024004,0.592899,0.071400,0.328428,-0.052907,2.883987e+05,48.400337,48.996046,...,0.495929,0.449413,0.736842,-0.001670,-0.001187,-0.165363,2015-05-15,ES,0.088235,1.0
AES/2017-01-27,0.202724,0.294530,0.032140,0.230590,0.073662,0.083645,-0.032021,-2.424996e+06,11.360934,11.554333,...,0.162176,0.314097,0.060606,-0.003697,-0.000341,-0.076259,2017-01-27,AES,0.717143,1.0
M/2016-07-18,0.265684,0.917763,0.043411,0.276355,0.122380,0.760138,0.103578,-1.856607e+06,34.755076,33.939910,...,0.406943,0.500084,0.295775,0.005432,-0.001134,0.110926,2016-07-18,M,0.307692,2.0


In [21]:
training_df.to_pickle('pickled_data/training_data.pkl')

In [22]:
training_df[training_df['Ticker'] == 'BRK']

,ADX_14_50,ATR_14,BollingerB_5,Bollinger%b_5,BollingerB_20,Bollinger%b_20,Copp_10,Chaikin,EMA_5,EMA_20,...,SO%d_5,SO%d_20,SO%k,Trix_5,Trix_20,TSI_25_13,Date,Ticker,TS,epsilon


In [23]:
training_df[training_df['Ticker'] == 'BF']

,ADX_14_50,ATR_14,BollingerB_5,Bollinger%b_5,BollingerB_20,Bollinger%b_20,Copp_10,Chaikin,EMA_5,EMA_20,...,SO%d_5,SO%d_20,SO%k,Trix_5,Trix_20,TSI_25_13,Date,Ticker,TS,epsilon
